## Import packages:

In [9]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
import gensim
import re
import unicodedata
import datetime
import collections
import random
import time
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import treetaggerwrapper
import string
import os
import platform
treetaggerPath = str(os.getcwd()) + "/treetagger/" + str(platform.system()) + "/"

In [2]:
treetaggerPath

'/home/francesco/Documents/workspace/gitprojects/TISProject/treetagger/Linux/'

## Load dataset:

In [3]:
trains24 = pd.read_csv("./Dataset/articoliS24O.csv", delimiter="\t")
trainRadiocor = pd.read_csv("./Dataset/articoliRadiocor.csv", delimiter='\t')

colsel_trains24 = trains24[['identificativo', 'body']]

colsel_trainRadiocor = trainRadiocor[['identificativo', 'body']]

colsel_merged = pd.concat([colsel_trains24, colsel_trainRadiocor])

colsel_merged.dropna(inplace=True)
colsel_merged = colsel_merged.reset_index()
colsel_merged = colsel_merged.drop('index', axis=1)

## Clean dataset:

In [4]:
print("SHAPE BEFORE REMOVING DUPLICATES: " + str(colsel_merged.shape)) 
colsel_merged.drop_duplicates(inplace=True) 
print("SHAPE AFTER REMOVING DUPLICATES: " + str(colsel_merged.shape))

colsel_merged['body'] = colsel_merged['body'].apply(lambda x : str(x).lower())
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(\\)+[a-z]\{[0-9]+\}','')
colsel_merged['body'] = colsel_merged['body'].str.replace('\n','')

#how to deal with times?
#colsel_merged['body'] = colsel_merged['body'].str.replace('[0-9]+\.[0-9]+','')

#handle percentages
colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(re.sub(r'([0-9]+)((\,|\.)[0-9]+)?((\s)?per(\s)?cento|(\s)?%)',r'\1_percento',str(x))))

#removing contracted articles and prepositions
colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(re.sub(r'[a-zA-Z]+\’',r' ',str(x))))
colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(re.sub(r'[a-zA-Z]+\'',r' ',str(x))))

colsel_merged['body'] = colsel_merged['body'].str.replace(r'(il)?( )?sole( )?24( )?ore','sole24ore')
colsel_merged['body'] = colsel_merged['body'].str.replace('(milion(i|e)|miliard(i|o)|euro)','')

#remove useless endings
colsel_merged['body'] = colsel_merged['body'].str.replace(r'\(c.fe.\)','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'((\()?( )?sole24ore (radiocor)?( plus)?( )?(\))?.\-.*?)\-','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(continua (da )?pagina [0-9]+)© riproduzione riservata(continua (da )?pagina [0-9]+)', '')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'((© )?riproduzione riser(vata)?).*','')

#format urls
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(www\.|http\:\/\/|https\:\/\/|\.com|\.net|\.org|\.it|@[a-zA-Z]+)','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','')  

#format dots for pos_tagging, done in order to mess up with urls!
colsel_merged['body'] = colsel_merged['body'].str.replace(r':',' : ')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'[a-zA-Z]+\'',' ')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'.','. ')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'( )+',' ')

#replace abbreviation
colsel_merged['body'] = colsel_merged['body'].str.replace('p/e','price_earnings_ratio')

#adjusted html tags cleaning! 2 is the magic number
for _ in np.arange(2):
    colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(BeautifulSoup(x,"lxml").text))
    
#escaping &
colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(re.sub(r'( )?\&( )?','and',str(x))))


#removing unprintable unicode
colsel_merged['body'].replace('\u200e','',regex=True,inplace=True)

SHAPE BEFORE REMOVING DUPLICATES: (10150, 2)
SHAPE AFTER REMOVING DUPLICATES: (9283, 2)


In [5]:
tmp= colsel_merged[colsel_merged['identificativo']=='FM20151009036ACmwyfCB']['body'].values
print(tmp)

[ 'pietro scott jovane lascia la poltrona di amministratore delegato di rcsmedia group. al termine di una giornata fitta di incontri e partita in prima mattinata con il summit con le banche creditrici per la rinegoziazione del debito, il consiglio di amministrazione del gruppo editoriale ha comunicato uscita «consensuale» di scott jovane a partire dal prossimo 15 ottobre e assegnazione delle deleghe di jovane ad interim al presidente maurizio costa. «il cda di rcs mediagroup - si legge in una nota - ha preso atto della volontà ad pietro scott jovane di lasciare il proprio incarico, ritenendo concluso un ciclo della vita aziendale e manifestando intenzione di intraprendere un nuovo percorso professionale».  inoltre il consiglio ha comunicato che per la risoluzione del contratto saranno corrisposte 150 mila al manager, a cui si sommano 600 mila lordi per il patto di non concorrenza. il board, iniziato alle 17. 30 in via rizzoli, nella sede centrale di rcs mediagroup, dopo la riunione del

In [6]:
print(colsel_merged[colsel_merged['identificativo']=='FM20160827022ADDGKkAB']['body'].values)

[ 'la saga continua. nonostante che tutti sappiano come va a finire. ma il commissario peo che si occupa di policy digitale, günther oettinger, non ne è al corrente, evidentemente. la saga è quella degli editori che si sentono defraudati di un diritto a causa dei motori di ricerca - cioè di google - che propongono ai lettori qualche riga dei loro articoli insieme ai link per trovarli online. oettinger sta studiando una proposta di direttiva che dà agli editori il diritto di chiedere un compenso per quelle righe di testo. forse pensa così di apparire come eroe della saga, innovatore che risolve un problema a lungo dibattuto. invece, arriva ultimo. e dopo che questo genere di soluzione è già fallito in spagna e in germania. ma è veramente possibile che non lo sappia? e il vice presidente della commissione, andrus ansip, che conosce il digitale molto meglio di oettinger e che si occupa del mercato unico digitale, non gli ha detto nulla? in realtà, la realtà digitale va troppo veloce per i

In [7]:
def clean_tag(tt):
    if(len(tt)<3):
        print(tt[0])
    if(tt[2]=='@card@' or tt[2]=='@ord@'):
        return tt[0]
    else:
        return re.sub(r'[a-z]+\|','',tt[2])

In [11]:
def pos_tagger_ita(lst):
    corpus = {}
    stopw = set(stopwords.words("italian"))
    puntk = set(string.punctuation)
    rem = stopw | puntk | set('«') | set('»')
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='it', TAGDIR=treetaggerPath)
    for txt in lst:
        tags=tagger.tag_text(txt[1])
        pp_tags= treetaggerwrapper.make_tags(tags)
        tagged_txt = [clean_tag(t) for t in pp_tags if clean_tag(t) not in rem]
        corpus[txt[0]]= tagged_txt
    return corpus

t1= time.time()
corp = pos_tagger_ita(colsel_merged[['identificativo','body']].values.tolist())
print('Lemmatisation done in '+ str((time.time()-t1)/60))

Lemmatisation done in 1.4114584565162658


In [12]:
corp['FM20151108016ACcDasOB']

['mondo',
 'biotecnologia',
 'investire',
 'bb',
 'biotech',
 'essere',
 'complesso',
 'comparto',
 'parte',
 'richiedere',
 'specializzazione',
 'altro',
 'seppure',
 'diverso',
 'società',
 'essere',
 'ormai',
 'consolidare',
 'mantenere',
 'caratteristica',
 'frontiera',
 'scientifico',
 'punto',
 'vista',
 'borsistico',
 'andare',
 'maneggiare',
 'cura',
 'certo',
 'lungo',
 'periodo',
 'potenzialità',
 'essere',
 'concretizzare',
 'nasdaq',
 'biotechnology',
 'index',
 'nbi',
 'cioè',
 'paniere',
 'essere',
 'quotare',
 'grande',
 'parte',
 'società',
 'settore',
 'ultimo',
 '10',
 'avere',
 'mettere',
 'segno',
 'incremento',
 'senza',
 'contare',
 're-investimento',
 'cedola',
 '335_percento',
 'chiusura',
 '3',
 '11',
 '2014',
 'sandp500',
 'invece',
 'stesso',
 'arco',
 'tempo',
 'contabilizzare',
 'dividendo',
 'essere',
 'salire',
 '105_percento',
 'insomma',
 'migliore',
 'andamento',
 'nbi',
 'rispetto',
 'wall',
 'street',
 'tradizionale',
 'essere',
 'numero',
 'ciò',
 '

In [13]:
corp['FM20151009036ACmwyfCB']

['pietro',
 'scott',
 'jovane',
 'lasciare',
 'poltrona',
 'amministratore',
 'delegare',
 'rcsmedia',
 'group',
 'termine',
 'giornata',
 'fitto',
 'incontro',
 'partita',
 'primo',
 'mattinata',
 'summit',
 'banca',
 'creditore',
 'rinegoziazione',
 'debito',
 'consiglio',
 'amministrazione',
 'gruppo',
 'editoriale',
 'avere',
 'comunicare',
 'uscire',
 'consensuale',
 'scott',
 'jovane',
 'partire',
 'prossimo',
 '15',
 'ottobre',
 'assegnazione',
 'delega',
 'jovane',
 'interim',
 'presidente',
 'maurizio',
 'costa',
 'cda',
 'rcs',
 'mediagroup',
 'leggere',
 'nota',
 'avere',
 'prendere',
 'atto',
 'volontà',
 'pietro',
 'scott',
 'jovane',
 'lasciare',
 'proprio',
 'incarico',
 'ritenere',
 'concludere',
 'ciclo',
 'vita',
 'aziendale',
 'manifestare',
 'intenzione',
 'intraprendere',
 'nuovo',
 'percorso',
 'professionale',
 'inoltre',
 'consiglio',
 'avere',
 'comunicare',
 'risoluzione',
 'contratto',
 'essere',
 'corrispondere',
 '150',
 'mila',
 'manager',
 'sommare',
 '60

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
# Initialize and train the model (this will take some time)
from gensim.models import word2vec

In [ ]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG='it', TAGDIR=treetaggerPath)
tags=tagger.tag_text('l\'articolo 4')
treetaggerwrapper.make_tags(tags)

In [ ]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 20          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [ ]:
print("Training model...")
model = word2vec.Word2Vec(list(corp.values()), workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

In [ ]:
model.most_similar("quotazione")

In [ ]:
model.most_similar("londra")

In [ ]:
model.syn0.shape

In [ ]:
model.most_similar("biotecnologia")